In [4]:
import pandas as pd
import numpy as np

In [6]:
data = pd.read_csv('F:\Workspace\data\자연어처리/credits.csv', low_memory=False)

In [7]:
movie = pd.read_csv('data/movies_metadata.csv', low_memory=False)

In [10]:
data.head(2)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [5]:
data.cast = data.cast.apply(lambda x: eval(x))

In [6]:
data.crew = data.crew.apply(lambda x: eval(x))

In [7]:
#df['clean_doc'] = df.overview.apply(lambda x: re.sub('[^A-Za-z ]', '', x))#구두점만 제거

In [8]:
def find_d(x):
    try:
        return x[0]['name']
    except:
        pass

In [9]:
find_d(data.crew[0])

'John Lasseter'

In [10]:
df = movie[['title','overview']]

In [11]:
df['감독'] = data.crew.apply(find_d)

C:\Users\TH\anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [12]:
df['주연'] = data.cast.apply(find_d)

In [13]:
df.tail()

,title,overview,감독,주연
45461,Subdue,Rising and falling between a man and woman.,Georges Méliès,Jeanne d'Alcy
45462,Century of Birthing,An artist struggles to finish his work while a...,Robert Fulton,None
45463,Betrayal,"When one of her hits goes wrong, a professiona...",A. R. Rahman,Sridevi Kapoor
45464,Satan Triumphant,"In a small town live two brothers, one a minis...",Shanra J. Kehl,Markie Adams
45465,Queerama,50 years after decriminalisation of homosexual...,Leo Tolstoy,Giulio Brogi


In [14]:
df.isnull().sum()

title          6
overview     954
감독           771
주연          2417
dtype: int64

In [15]:
df.dropna(how='any', inplace=True) # how = 'any'가 디폴트

C:\Users\TH\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [16]:
df.set_index('title',inplace=True)
df.reset_index(inplace=True)
df.tail()

,title,overview,감독,주연
41829,Robin Hood,"Yet another version of the classic epic, with ...",Georges Méliès,Georges Méliès
41830,Subdue,Rising and falling between a man and woman.,Georges Méliès,Jeanne d'Alcy
41831,Betrayal,"When one of her hits goes wrong, a professiona...",A. R. Rahman,Sridevi Kapoor
41832,Satan Triumphant,"In a small town live two brothers, one a minis...",Shanra J. Kehl,Markie Adams
41833,Queerama,50 years after decriminalisation of homosexual...,Leo Tolstoy,Giulio Brogi


In [17]:
df = df.head(20000)

In [18]:
df

,title,overview,감독,주연
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",John Lasseter,Tom Hanks
1,Jumanji,When siblings Judy and Peter discover an encha...,Larry J. Franco,Robin Williams
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,Howard Deutch,Walter Matthau
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",Forest Whitaker,Whitney Houston
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,Alan Silvestri,Steve Martin
...,...,...,...,...
19995,A Better Way to Die,Boomer (Wiper) is a rookie cop based in the ur...,Hal Kanter,Bob Hope
19996,Casanova's Big Night,"Italy 1757, Pippo Popolino, a lowly tailor, di...",Meg Morman,Michael Paré
19997,Tunnel Rats,During the Vietnam War [1959-1975] a special U...,Jules Verne,Michael Caine
19998,"20,000 Leagues Under the Sea","The year is 1886, when New England's fishing h...",John G. Adolfi,George Arliss


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvect = TfidfVectorizer(stop_words='english')
tfidf_matrix = tvect.fit_transform(df.overview,df)
tfidf_matrix.shape

(20000, 47435)

In [20]:
indices = pd.Series(df.index, index=df.title).drop_duplicates()
indices.head() #숫자형태로 바꾸었다 일부러

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [21]:
indices.shape

(20000,)

In [22]:
from sklearn.metrics.pairwise import linear_kernel

In [23]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [24]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return df[['title', '감독', '주연']].iloc[movie_indices]

In [60]:
def get_recommend_actor(title):
        idx = indices[title]
        mainactor = df.iloc[idx].주연
        return df.set_index('주연').T[df.iloc[idx].주연].iloc[0][:10]

In [53]:
def get_recommend_director(title):
    idx = indices[title]
    direcotr = df.iloc[idx].감독
    return df.set_index('감독').T[df.iloc[idx].감독].iloc[0][:10]


In [25]:
get_recommendations('The Dark Knight Rises')


,title,감독,주연
12229,The Dark Knight,Lucinda Syson,Christian Bale
148,Batman Forever,Joel Schumacher,Val Kilmer
1289,Batman Returns,Tim Burton,Michael Keaton
15107,Batman: Under the Red Hood,Bob Kane,Bruce Greenwood
578,Batman,Ray Lovejoy,Jack Nicholson
9049,Batman Beyond: Return of the Joker,Michael Uslan,Will Friedle
17454,Batman: Year One,Frank Miller,Ben McKenzie
19058,"Batman: The Dark Knight Returns, Part 1",Costa-Gavras,Gad Elmaleh
3027,Batman: Mask of the Phantasm,Michael Uslan,Kevin Conroy
19455,"Batman: The Dark Knight Returns, Part 2",Amos Poe,Vincent Spano


In [61]:
get_recommend_actor('The Dark Knight Rises')


주연
Christian Bale              Newsies
Christian Bale            Metroland
Christian Bale      American Psycho
Christian Bale    Empire of the Sun
Christian Bale        Reign of Fire
Christian Bale          Equilibrium
Christian Bale        The Machinist
Christian Bale        Batman Begins
Christian Bale          Harsh Times
Christian Bale          Rescue Dawn
Name: title, dtype: object

In [55]:
get_recommend_director('The Dark Knight Rises')

감독
Hans Zimmer              Drop Zone
Hans Zimmer        Renaissance Man
Hans Zimmer             Two Deaths
Hans Zimmer          Cool Runnings
Hans Zimmer    The Preacher's Wife
Hans Zimmer               Rain Man
Hans Zimmer     Driving Miss Daisy
Hans Zimmer      The Thin Red Line
Hans Zimmer              Backdraft
Hans Zimmer         Bird on a Wire
Name: title, dtype: object

In [2]:
idx = indices['The Dark Knight Rises']
df.iloc[idx].감독
df.set_index('감독').T[df.iloc[idx].감독].iloc[0][:10]


NameError: name 'indices' is not defined